# Medallion Architecture Data Cleaning Pipeline

Delta Live Tables offer a fault-tolerant, optimized approach for building reliable data pipelines, making them ideal for this use case.

In the real world, roles & responsibilities of E2E data projects are as shown:

- **Data Engineers**: Focus on building pipelines that handle common data issues such as duplicates, formatting of columns, schema definition, and invalid values.

- **Data Scientists**: Work on EDA, imputing missing values, handling outliers, and preparing data for modeling (feature engineering / selection / dimensionality reduction etc).

In this notebook, I will be implementing a simplified **Medallion Architecture** using **Delta Live Tables (DLT)
in Azure Databricks** to simulate real-world data engineering practices.

I will be using the following visualisation as a guide to build the data pipeline.

<br>

<img src="https://media.datacamp.com/cms/ad_4nxe4oejrhu9gexxri3ea6vmsu1fgxcxbvlwmbaj4ji5s2u31dg3hbyyg4sxmd7ma8-9zamnbxadzz_h4kllvjylicug3v4-iinvx65erdijn4htymmqvc3mjqblskqzdu5ttmodyua.png">

By the end of this notebook, I should be able to:

- Output a **thoroughly cleansed target dataset** ready for data scientists' to conduct EDA, dataset preprocessing and other model building practices.

- Define **feature and target variables** from the target table clearly


## 0. Import Libraries


In [1]:
# == Import Libraries ==

from init_spark import start_spark

spark = start_spark()

from pyspark.sql.functions import (
    col,
    when,
    count,
    desc,
    isnan,
    isnull,
    lit,
    length,
    trim,
    lower,
    upper,
    to_date,
    concat_ws,
    regexp_extract,
    expr,
)

from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    DoubleType,
    IntegerType,
    DateType,
    NumericType,
)

from pyspark.sql import DataFrame
from datetime import date

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/04 10:45:15 WARN Utils: Your hostname, Chengs-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.77 instead (on interface en0)
25/09/04 10:45:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/lunlun/Downloads/Github/Credit-Risk-Modeling-PySpark/venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/lunlun/.ivy2.5.2/cache
The jars for the packages stored in: /Users/lunlun/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4f937c24-62d9-4620-94df-b72e1a67bdc0;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: resolution report :: 

4.0.0


In [2]:
# == Helper Functions (String Cleaning) ==


def drop_duplicates(df):
    duplicate_rows = df.count() - df.dropDuplicates().count()

    return df.dropDuplicates()


def handle_string_cols_spaces(df):
    string_cols = [
        field.name
        for field in df.schema.fields
        if isinstance(field.dataType, StringType)
    ]

    # Replaces each existing column with new <string> values which are trimmed
    for col_name in string_cols:
        df = df.withColumn(col_name, trim(col(col_name)))

    return df


def handle_string_cols_formatting(df):
    """
    Uses library of RapidFuzz to provide lightweight similarity calculations, optimised for performance

    Takes reference from String issues are in ../sandbox/string_issues.ipynb
    """

    print(f"Original Number of Rows: {df.count()}. ")

    # # 1. Drops unusable String columns
    # unusable_cols = ["emp_title",
    # "hardship_type",
    # "verification_status_joint",
    # "hardship_status",
    # "deferral_term",
    # "hardship_length",
    # "hardship_loan_status",
    # "settlement_status", "annual_inc_joint", 'dti_joint']
    # df = df.drop(*unusable_cols) # allows dropping of multiple columns

    # 2. Fix addr_state (check len() > 2)
    df = df.filter(length(col("addr_state")) == 2)

    # 3. Fix invalid string column values

    # invalid_entries_list = [ "application_type",
    #     "policy_code",
    #     "home_ownership",
    #     "verification_status",
    #     "loan_status",
    #     "pymnt_plan",
    #     "initial_list_status",
    #     "hardship_flag"]

    df = df.filter(
        col("application_type").isin("Individual", "Joint App")
        & col("policy_code").isin("1.0", "0.0")  # stored as string initially
        & col("home_ownership").isin("MORTGAGE", "RENT", "OWN", "ANY", "OTHER")
        & col("verification_status").isin("Source Verified", "Not Verified", "Verified")
        & col("loan_status").isin("Fully Paid", "Charged Off", "Default")
        & col("pymnt_plan").isin("n", "y")
        & col("initial_list_status").isin("f", "w")
        & col("hardship_flag").isin("N", "Y")
    )
    df = df.withColumnRenamed("loan_status", "default_status")

    df = df.withColumn(
        "default_status",
        when(col("default_status") == "Fully Paid", 0)
        .when(col("default_status") == "Charged Off", 1)
        .when(col("default_status") == "Default", 1),
    )
    df = df.withColumn(
        "home_ownership",
        when(col("home_ownership") == "ANY", "OTHER").otherwise(col("home_ownership")),
    )

    df = df.withColumn(
        "policy_code",
        when(col("policy_code") == "1.0", 1).when(col("policy_code") == "0.0", 0),
    )

    # 4. Drop meaningless string columns
    meaningless_columns = ["url", "desc", "title", "zip_code", "purpose"]
    df = df.drop(*meaningless_columns)

    return df


def cast_string_to_numeric_cols(df):
    numeric_columns = [
        "id",
        "member_id",
        "annual_inc",
        "annual_inc_joint",
        "dti",
        "dti_joint",
        "delinq_2yrs",
        "fico_range_low",
        "fico_range_high",
        "inq_last_6mths",
        "mths_since_last_delinq",
        "mths_since_last_record",
        "open_acc",
        "pub_rec",
        "revol_bal",
        "revol_util",
        "total_acc",
        "out_prncp",
        "out_prncp_inv",
        "total_pymnt",
        "total_pymnt_inv",
        "total_rec_prncp",
        "total_rec_int",
        "total_rec_late_fee",
        "recoveries",
        "collection_recovery_fee",
        "last_fico_range_high",
        "last_fico_range_low",
        "collections_12_mths_ex_med",
        "mths_since_last_major_derog",
        "acc_now_delinq",
        "tot_coll_amt",
        "loan_amnt",
        "funded_amnt",
        "funded_amnt_inv",
        "installment",
        "tot_cur_bal",
        "total_rev_hi_lim",
        "inq_fi",
        "hardship_amount",
        "hardship_dpd",
        "orig_projected_additional_accrued_interest",
        "hardship_payoff_balance_amount",
        "hardship_last_payment_amount",
        "settlement_amount",
        "settlement_percentage",
        "settlement_term",
        "avg_cur_bal",
        "total_bal_il",
        "bc_util",
        "il_util",
        "total_cu_tl",
        "max_bal_bc",
        "percent_bc_gt_75",
        "total_bal_ex_mort",
        "all_util",
        "open_acc_6m",
        "open_act_il",
        "open_il_12m",
        "last_pymnt_amnt",
        "open_il_24m",
        "mths_since_rcnt_il",
        "open_rv_12m",
        "open_rv_24m",
        "emp_length",
        "term",
    ]

    # Deal with Type Casting to Numeric Data
    int_cols = [
        "id",
        "member_id",
        "delinq_2yrs",
        "fico_range_low",
        "fico_range_high",
        "inq_last_6mths",
        "mths_since_last_delinq",
        "mths_since_last_record",
        "open_acc",
        "pub_rec",
        "total_acc",
        "last_fico_range_high",
        "last_fico_range_low",
        "collections_12_mths_ex_med",
        "mths_since_last_major_derog",
        "acc_now_delinq",
        "inq_fi",
        "hardship_dpd",
        "settlement_term",
        "total_cu_tl",
        "open_acc_6m",
        "open_act_il",
        "open_il_12m",
        "open_il_24m",
        "open_rv_12m",
        "open_rv_24m",
        "emp_length",
        "mths_since_rcnt_il",
    ]

    for column in numeric_columns:
        if column == "emp_length":
            # == Converts emp_length to integer value ==
            df = df.withColumn(
                "emp_length",
                when(col("emp_length").rlike("10\\+"), 10)
                .when(col("emp_length").rlike("< 1"), 0)
                .otherwise(
                    regexp_extract(col("emp_length"), r"(\d+)", 1).cast(
                        "int"
                    )  # extracts 1st regex group digit from string
                ),
            )

        elif column in int_cols:
            df = df.withColumn(
                column, col(column).cast(DoubleType()).cast(IntegerType())
            )

        elif column == "term":
            df = df.withColumn(
                "term",
                regexp_extract(col("term"), r"(\d+)", 1)
                .cast(DoubleType())
                .cast(IntegerType()),
            )

        else:
            df = df.withColumn(column, col(column).cast(DoubleType()))

    return df


def cast_string_to_date_cols(df):
    date_columns = [
        "issue_d",
        "earliest_cr_line",
        "last_pymnt_d",
        "last_credit_pull_d",
        "next_pymnt_d",
        "sec_app_earliest_cr_line",
        "hardship_start_date",
        "hardship_end_date",
        "payment_plan_start_date",
        "debt_settlement_flag_date",
        "settlement_date",
    ]

    # Clean and cast each column
    for date_col in date_columns:
        # Only parse values that match the pattern like "Jan-2015"
        df = df.withColumn(
            date_col,
            when(
                col(date_col).rlike("^[A-Za-z]{3}-\\d{4}$"),  # only parse valid ones
                to_date(concat_ws("-", col(date_col), lit("01")), "MMM-yyyy-dd"),
            ).otherwise(None),
        )  # fallback for malformed ones

    return df


# == Helper Functions (Numerical Cleaning)
def clear_invalid_numerical_entries(df):
    df = df.filter(~(col("dti") < 0))
    df = df.filter(~(col("total_rec_late_fee") < 0))
    df = df.filter(col("loan_amnt") > 0)
    df = df.filter(col("funded_amnt") > 0)
    df = df.filter(
        (col("int_rate") >= 0) & (col("int_rate") <= 100)
    )  # Assuming interest rate is between 0% and 100%
    df = df.filter(col("installment") >= 0)
    df = df.filter(col("annual_inc") >= 0)
    df = df.filter(
        (col("revol_util") >= 0) & (col("revol_util") <= 100)
    )  # Else, user maxed out credit accounts' limits
    df = df.filter(col("total_rec_late_fee") >= 0)
    df = df.filter(col("recoveries") >= 0)
    df = df.filter(col("collection_recovery_fee") >= 0)
    df = df.filter(col("total_rec_prncp") >= 0)
    df = df.filter(col("total_rec_int") >= 0)
    df = df.filter(col("total_pymnt") >= 0)
    df = df.filter(col("out_prncp") >= 0)

    # Fico Range Invalid Entries Removal
    df = df.filter((col("fico_range_low") >= 300) & (col("fico_range_low") <= 850))
    df = df.filter((col("fico_range_high") >= 300) & (col("fico_range_high") <= 850))
    df = df.filter(col("fico_range_low") <= col("fico_range_high"))

    return df


def drop_constant_columns(df):
    """
    Removes all columns in the DataFrame that have only one distinct value.
    Returns a new DataFrame with those columns removed.
    """
    cols_to_drop = []

    for column_name in df.columns:
        if df.select(col(column_name)).distinct().count() <= 1:
            cols_to_drop.append(column_name)

    print(f"⚠️ Dropping constant columns: {cols_to_drop}")
    return df.drop(*cols_to_drop)

## 1. Bronze Delta Table

This serves as a 'landing place' for raw data for single-source of truth purposes. In case data processing in subsequent stages go faulty, data specialists can use the **Bronze Delta Table** for reference, ensuring data integrity.


In [3]:
# import dlt (specific to databricks)

# This will only be allowed if I can create a DLT pipeline (not allowed due to Azure for Students)

# @dlt.table(name="bronze_raw_lendingclub_data", comment="Ingest raw loan data from Lending Club csv")
# def bronze_raw_loans():
#     return spark.read.csv("/FileStore/tables/accepted_2007_to_2018Q4.csv",
#                           header=True,
#                           inferSchema=True)

# I will need to ensure inferSchema = True, so that all columns dtypes are auto-detected to lessen my workload later

# ✅ == The below allows DLT pipeline not to be created ==

bronze_df = (
    spark.read.option("header", True)
    .option("inferSchema", True)
    .csv("../data/accepted_2007_to_2018Q4.csv")
)


# ✅ 2. Save as a Delta table in the `bronze` schema
bronze_df.write.format("delta").mode("overwrite").save("../data/bronze/lendingclub_raw")

25/09/04 10:45:25 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


## 2. Silver Delta Table

Next, the pipeline to produce a Silver Delta Table will mainly perform key data cleaning steps.

- Deal with Duplicates
- Remove String Column Spaces
- Handle String Formatting / Spelling Issues
- Ensure UTF-8 for String Columns
- Schema Definition
- Invalid Value Handling


### 2.1 String Columns Cleaning

These cleaning steps will take reference from **sandbox/string_issues** for specific cleaning steps to maintain data integrity.


In [4]:
# == Step 0: Read from Bronze Table ==
bronze_df_copy = spark.read.format("delta").load("../data/bronze/lendingclub_raw")


# == Step 1: Drop duplicate rows ==
print(f"Original number of rows: {bronze_df_copy.count()}\n")

df_cleaned = drop_duplicates(bronze_df_copy)
print("✅ Duplicates removed...")


# == Step 2: Trim spaces in all string columns ==
df_cleaned = handle_string_cols_spaces(df_cleaned)
print("✅ Trailing / Leading Spaces removed...")

# == Step 3: Filter & fix invalid string formatting ==
df_cleaned = handle_string_cols_formatting(df_cleaned)
print("✅ Invalid String Column Formatting Settled & Meaningless Columns Dropped ...")

# == Step 4: Type Casting ==
df_cleaned = cast_string_to_numeric_cols(df_cleaned)
df_cleaned = cast_string_to_date_cols(df_cleaned)
print("✅ String Columns Correctly Type Casted...\n")

print(f"New number of rows: {df_cleaned.count()}")

# == Step 5: Save as Silver Delta Table 1 (Cleaned Strings Version) ==
df_cleaned.write.format("delta").mode("overwrite").save(
    "../data/silver/lendingclub_cleaned_string"
)

Original number of rows: 2260701



✅ Duplicates removed...
✅ Trailing / Leading Spaces removed...


Original Number of Rows: 2260701. 
✅ Invalid String Column Formatting Settled & Meaningless Columns Dropped ...
✅ String Columns Correctly Type Casted...



New number of rows: 1345075


In [5]:
df_cleaned.limit(10).toPandas()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,1165552,NaN,12000.0,12000.0,12000.0,36,6.62,368.45,A,A2,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
1,1375351,NaN,4000.0,4000.0,4000.0,36,7.62,124.65,A,A3,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
2,1342664,NaN,8875.0,8875.0,8875.0,36,10.74,289.47,B,B2,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
3,1460588,NaN,12000.0,12000.0,12000.0,36,8.90,381.04,A,A5,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
4,1294082,NaN,6000.0,6000.0,5900.0,36,6.03,182.62,A,A1,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
5,1209893,NaN,17600.0,17600.0,17350.0,60,12.12,392.58,B,B3,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
6,1304249,NaN,12000.0,12000.0,12000.0,36,13.67,408.22,B,B5,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
7,1136970,NaN,22650.0,22650.0,22650.0,60,16.29,554.31,C,C5,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
8,1187963,NaN,4000.0,4000.0,4000.0,36,8.90,127.02,A,A5,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
9,1116600,NaN,11000.0,11000.0,11000.0,36,12.12,365.99,B,B3,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN


### 2.2 Numeric Columns Cleaning

These cleaning steps will take reference from **sandbox/numeric_issues** for specific cleaning steps to maintain data integrity


In [6]:
# == Remove invalid numerical values in dataset ==
silver_table1 = spark.read.format("delta").load(
    "../data/silver/lendingclub_cleaned_string"
)


print(f"Original number of rows: {silver_table1.count()}\n")

silver_table2 = clear_invalid_numerical_entries(silver_table1)
print("✅ Invalid Numerical Values Settled...")

print(f"Final number of rows: {silver_table2.count()}\n")

Original number of rows: 1345075

✅ Invalid Numerical Values Settled...
Final number of rows: 1339155



In [7]:
silver_table2.limit(10).toPandas()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,670653,NaN,5000.0,5000.0,5000.000000,36,10.74,163.08,B,B4,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
1,843470,NaN,8500.0,8500.0,8500.000000,36,7.49,264.37,A,A4,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
2,695876,NaN,5000.0,5000.0,5000.000000,36,10.37,162.21,B,B3,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
3,766091,NaN,1875.0,1875.0,1875.000000,36,8.49,59.19,A,A5,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
4,505004,NaN,7000.0,7000.0,6062.370474,36,7.51,217.77,A,A4,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
5,476497,NaN,6250.0,6250.0,6225.000000,36,12.53,209.17,B,B5,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
6,1030372,NaN,8000.0,8000.0,8000.000000,36,13.49,271.45,C,C1,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
7,543747,NaN,5000.0,5000.0,5000.000000,36,10.75,163.11,B,B2,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
8,1377853,NaN,20000.0,20000.0,20000.000000,36,13.99,683.46,C,C1,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN
9,371976,NaN,1000.0,1000.0,209.198360,36,14.74,34.54,D,D3,...,NaN,NaN,Cash,N,None,None,None,NaN,NaN,NaN


### 2.3 Date Column Cleaning


In [8]:
# == Change earliest_cr_line column to credit_history_years (Derived Column) ==
from pyspark.sql.functions import months_between, col

silver_table2 = silver_table2.withColumn(
    "credit_history_years",
    (months_between(col("issue_d"), col("earliest_cr_line")) / 12).cast("int"),
)
silver_table2 = silver_table2.drop(col("earliest_cr_line"))

### 2.4 Handling Missing Values

Based on `../sandbox/string_issues.ipynb`, I have identified the following issues with data quality.

- Some columns are unusable, due to to having a **large percentage of missing values** (taken reference from `../sandbox/string_issues`)

- A number of features exhibit **missing values**, including `emp_length` etc. This requires imputation strategies to be implemented

- Some columns are irrelevant to our credit risk modeling project, due to **high cardinality** (large number of unique values), e.g. `emp_title` **(categorical data)**.

- There are also **redundant columns**, like `member_id`, which provides no value to our prediction of LGD, EAD and PD.

- There are features like `delinq_2yrs` which have **outliers** (maximum data point way above the 75% quartile)

#### 2.4.1 Find Null Value % Per Column

For this credit risk modeling project, I will be dropping columns with &gt; 50% missingness. Many credit risk modelling projects on Kaggle and Github use 50%-65% missingness as the threshold to drop columns as well.

However, let's display the columns which have >=50% missing values first to inspect them.


In [9]:
from pyspark.sql.functions import col, sum

# == Get total number of rows ==
total_rows = silver_table2.count()

# == Calculate % of nulls per column and keep only those ≥ 50% ==
missing_val_threshold = 30
high_missingness_columns = []

for column in silver_table2.schema.fields:
    null_count = silver_table2.select(
        sum(col(column.name).isNull().cast("int"))
    ).collect()[0][0]
    null_pct = (null_count / total_rows) * 100
    if null_pct >= missing_val_threshold:
        print(f"{column.name}: {null_pct:.2f}% null")
        high_missingness_columns.append(column.name)

# == Drop columns with >= 50% missing values (Low predictive power upon inspection) ==
silver_table2 = silver_table2.drop(*high_missingness_columns)
print("\n✅ Columns with high pct of missing values dropped ... \n")

# == Inspect Dimensions ==
num_rows = silver_table2.count()
num_cols = len(silver_table2.columns)
print(f"Updated Shape: ({num_rows}, {num_cols})")

member_id: 100.00% null


mths_since_last_delinq: 50.43% null
mths_since_last_record: 82.96% null


next_pymnt_d: 100.00% null
mths_since_last_major_derog: 73.68% null
annual_inc_joint: 98.11% null
dti_joint: 98.11% null
verification_status_joint: 98.12% null
open_acc_6m: 60.04% null
open_act_il: 60.04% null
open_il_12m: 60.04% null
open_il_24m: 60.04% null
mths_since_rcnt_il: 61.09% null
total_bal_il: 60.04% null
il_util: 65.43% null
open_rv_12m: 60.04% null
open_rv_24m: 60.04% null
max_bal_bc: 60.04% null
all_util: 60.04% null
inq_fi: 60.04% null
total_cu_tl: 60.04% null
inq_last_12m: 60.04% null
mths_since_recent_bc_dlq: 76.28% null
mths_since_recent_revol_delinq: 66.54% null
revol_bal_joint: 98.64% null
sec_app_fico_range_low: 98.64% null
sec_app_fico_range_high: 98.64% null
sec_app_earliest_cr_line: 98.64% null
sec_app_inq_last_6mths: 98.64% null
sec_app_mort_acc: 98.64% null
sec_app_open_acc: 98.64% null
sec_app_revol_util: 98.66% null
sec_app_open_act_il: 98.64% null
sec_app_num_rev_accts: 98.64% null
sec_app_chargeoff_within_12_mths: 98.64% null
sec_app_collections_12_mths_ex

#### 2.4.2 Dropping Irrelevant / Redundant Columns

This section implements the removal of **meaningless columns, features which has high cardinality (categorical data), features with little predictive value, e.g. `member_id`, `emp_title` etc, and post-loan features**. Including such features may lead to multicollinearity, and ultimately lead to low predictive power of our credit models.

Reasons why I removed certain columns are as shown:

- Columns with `inv`: Largely same as its subset, e.g. `total_pymnt_inv` is largely the same as `total_pymnt`

- `last_pymnt_d` and `last_credit_pull_d` (according to Data Dictionary) have little predictive value even after feature engineering. It merely shows the last payment date by borrower and last date where credit report is pulled. This has little value in predicting PD, LGD or EAD.

- `sub_grade` is more granular than `grade`. This may lead to a risk of overfitting of our PD, LGD, and EAD models.

- High Cardinality Columns may lead to high computational costs in encoding for machine learning models, which makes it undesirable in a big data space such as credit risk.

- Hardship & Settlement Features (Borrowers are only eligible for hardship and settlement programmes after loan origination for Lending Club, not when they apply for it). Borrowers will contact lenders of financial hardship, attempting to settle with lenders for interest-fee payments or lower principal sum payments. Such features should not be used to predict PD, LGD, and EAD. My models should not know if a borrower will fall into hardship for this credit risk modeling project

- `disbursement_method` indicates how loan funds are delivered to the borrower. This has little relevance in predicting PD, LGD or EAD.


In [10]:
# == Drop Derived/Meaningless Features ==
derived_features = [
    "funded_amnt_inv",
    "sub_grade",
    "out_prncp_inv",
    "total_pymnt_inv",
    "last_pymnt_d",
    "last_credit_pull_d",
]
silver_table2 = silver_table2.drop(*derived_features)
print(f"✅ Derived/Meaningless Features Dropped ...")

✅ Derived/Meaningless Features Dropped ...


In [11]:
# == Drop High Cardinality Features ==

# == 1. Define Threshold ==
high_cardinality_threshold = 50

# == 2. Find Categorical Features (to identify high cardinality columns) ==
categorical_cols = [
    field.name
    for field in silver_table2.schema.fields
    if isinstance(field.dataType, StringType)
]
print(categorical_cols)

# == 3. Identify high-cardinality columns ==
high_card_cols = []

for col_name in categorical_cols:
    unique_count = silver_table2.select(col_name).distinct().count()

    if unique_count >= high_cardinality_threshold:
        print(f"\n{col_name} has {unique_count} unique values → dropping ... ")
        high_card_cols.append(col_name)

# == 4. Drop high cardinality columns ==
silver_table2 = silver_table2.drop(*high_card_cols)

print(f"\n✅ High Cardinality Features Dropped ...")

['grade', 'emp_title', 'home_ownership', 'verification_status', 'pymnt_plan', 'addr_state', 'initial_list_status', 'application_type', 'hardship_flag', 'disbursement_method', 'debt_settlement_flag']

emp_title has 359391 unique values → dropping ... 

addr_state has 51 unique values → dropping ... 

✅ High Cardinality Features Dropped ...


In [12]:
# ==  Drop columns with only 1 distinct value (No variation is useless for modeling phase)==
silver_table2 = drop_constant_columns(silver_table2)
print("✅ Columns with only 1 distinct value dropped...")

⚠️ Dropping constant columns: ['pymnt_plan', 'policy_code', 'hardship_flag']
✅ Columns with only 1 distinct value dropped...


In [13]:
# == Drop hardship related columns & miscelleanous columns (rare & irrelevant for modeling) ==
hardship_columns = [
    "hardship_flag",
    "disbursement_method",
    "debt_settlement_flag",
    "policy_code",
]

silver_table2 = silver_table2.drop(*hardship_columns)
print("✅ Hardship & miscelleneous columns dropped ...")

✅ Hardship & miscelleneous columns dropped ...


#### 2.4.3 Impute Missing Values (Categorical & Numerical)

After removing unnecessary columns with little predictive power, we will proceed to impute missing values. We will first identify % missing values per column.

For numerical columns, median values shall replace missing values, given how we haven't dealt with outliers yet. For categorical columns, mode categories shall be used to replace missing values. Such an approach is common and simplistic, though there are advanced imputation techniques like clustering. However, we shall not lose focus of learning about the credit risk modeling domain in this project.


In [14]:
total_rows = silver_table2.count()

for column in silver_table2.columns:
    null_count = silver_table2.filter(col(column).isNull()).count()
    if null_count > 0:
        print(
            f"{column}: {null_count} null values, {round(null_count/total_rows * 100,2)}% missing values."
        )

emp_length: 77891 null values, 5.82% missing values.
inq_last_6mths: 1 null values, 0.0% missing values.
collections_12_mths_ex_med: 54 null values, 0.0% missing values.
tot_coll_amt: 67219 null values, 5.02% missing values.
tot_cur_bal: 67219 null values, 5.02% missing values.
total_rev_hi_lim: 67219 null values, 5.02% missing values.
acc_open_past_24mths: 46992 null values, 3.51% missing values.
avg_cur_bal: 67219 null values, 5.02% missing values.
bc_open_to_buy: 59990 null values, 4.48% missing values.
bc_util: 60631 null values, 4.53% missing values.
chargeoff_within_12_mths: 54 null values, 0.0% missing values.
mo_sin_old_il_acct: 105039 null values, 7.84% missing values.
mo_sin_old_rev_tl_op: 67219 null values, 5.02% missing values.
mo_sin_rcnt_rev_tl_op: 67219 null values, 5.02% missing values.
mo_sin_rcnt_tl: 67219 null values, 5.02% missing values.
mort_acc: 46992 null values, 3.51% missing values.
mths_since_recent_bc: 59142 null values, 4.42% missing values.
mths_since_rece

In [15]:
# == Loop over each column ==
for feature in silver_table2.schema.fields:
    col_name = feature.name
    dtype = feature.dataType

    if isinstance(dtype, StringType):
        mode_value = (
            silver_table2.groupBy(col(f"{col_name}"))
            .count()
            .orderBy(col("count").desc())
            .first()[0]
        )

        silver_table2 = silver_table2.fillna({f"{col_name}": mode_value})

    # == Impute Numerical Columns with Median ==
    elif isinstance(dtype, NumericType):
        if silver_table2.filter(col(col_name).isNull()).count() > 0:
            median_val = silver_table2.approxQuantile(col_name, [0.5], 0.01)[0]
            silver_table2 = silver_table2.fillna({col_name: median_val})

print("✅ Categorical Column Missing Values Filled!")
print("✅ Numerical Column Missing Values Filled!")

✅ Categorical Column Missing Values Filled!
✅ Numerical Column Missing Values Filled!


In [16]:
# == Double check if there are any missing values before subsequent steps ==
total_rows = silver_table2.count()

output_arr = []
for column in silver_table2.columns:
    null_count = silver_table2.filter(col(column).isNull()).count()
    if null_count > 0:
        output_arr.append(
            f"{column}: {null_count} null values, {round(null_count/total_rows * 100,2)}% missing values."
        )

if len(output_arr) == 0:
    print("✅ No Missing Values Found!")
else:
    print(output_arr)

✅ No Missing Values Found!


#### 2.4.5 Base Feature Creation

Let's obtain some base features from our dataset.

First, a common derived feature in credit risk is the credit history length of the borrower at the time of loan issuance. Normally, it would be safe to say that loans with borrowers with a longer credit history has a lower probability of default. Hence, this can be taken into account as 1 of the features of the dataset. We can compute this as the difference between `issue_d` and `earliest_cr_line`.

Second, it seems that we have 2 features related to FICO Scores: `fico_range_low` and `fico_range_high`. Upon further research, these features do not represent the lowest and highest scores ever recorded for a borrower, but rather the lower and upper bounds of a small range in which the borrower’s actual FICO score falls. Banks record these due to privacy concerns, through a 5 point window, as shown below. As such, it seems that we just need to average these 2 values, to minimise redundancy and multicollinearity.


In [17]:
silver_table2 = silver_table2.withColumn(
    "fico_score", (col("fico_range_low") + col("fico_range_high")) / 2
)
silver_table2 = silver_table2.drop(col("fico_range_low"), col("fico_range_high"))

# == Assert checks ==
expected_columns = set(silver_table2.columns)

# == 1. Confirm 'fico_score' exists ==
assert "fico_score" in expected_columns, "'fico_score' column is missing"

# == 2. Confirm 'fico_range_low' and 'fico_range_high' are removed ==
assert (
    "fico_range_low" not in expected_columns
    and "fico_range_high" not in expected_columns
), "Old FICO range columns still present"

In [18]:
# == Save as Silver Delta Table 2 (Cleaned Strings & Numbers Version) ==
silver_table2.write.format("delta").mode("overwrite").save(
    "../data/silver/lendingclub_cleaned_numeric"
)

silver_table2.limit(10).toPandas()

,id,loan_amnt,funded_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,credit_history_years,fico_score
0,670653,5000.0,5000.0,36,10.74,163.08,B,6,MORTGAGE,35184.0,...,98.0,42.9,0.0,0.0,111168.0,37033.0,15000.0,31463.0,5,687.0
1,843470,8500.0,8500.0,36,7.49,264.37,A,6,MORTGAGE,230000.0,...,98.0,42.9,0.0,0.0,111168.0,37033.0,15000.0,31463.0,20,742.0
2,695876,5000.0,5000.0,36,10.37,162.21,B,6,RENT,17220.0,...,98.0,42.9,1.0,0.0,111168.0,37033.0,15000.0,31463.0,12,697.0
3,766091,1875.0,1875.0,36,8.49,59.19,A,6,RENT,15204.0,...,98.0,42.9,0.0,0.0,111168.0,37033.0,15000.0,31463.0,11,722.0
4,505004,7000.0,7000.0,36,7.51,217.77,A,2,RENT,38400.0,...,98.0,42.9,0.0,0.0,111168.0,37033.0,15000.0,31463.0,10,737.0
5,476497,6250.0,6250.0,36,12.53,209.17,B,0,RENT,24000.0,...,98.0,42.9,0.0,0.0,111168.0,37033.0,15000.0,31463.0,5,697.0
6,1030372,8000.0,8000.0,36,13.49,271.45,C,10,MORTGAGE,98000.0,...,98.0,42.9,0.0,0.0,111168.0,37033.0,15000.0,31463.0,11,672.0
7,543747,5000.0,5000.0,36,10.75,163.11,B,9,MORTGAGE,46000.0,...,98.0,42.9,0.0,0.0,111168.0,37033.0,15000.0,31463.0,21,712.0
8,1377853,20000.0,20000.0,36,13.99,683.46,C,2,MORTGAGE,75000.0,...,98.0,37.5,0.0,0.0,111168.0,17517.0,49400.0,31463.0,15,682.0
9,371976,1000.0,1000.0,36,14.74,34.54,D,0,RENT,10000.0,...,98.0,42.9,0.0,0.0,111168.0,37033.0,15000.0,31463.0,5,667.0


In [19]:
silver_table2.limit(10).toPandas()

print("✅ All missing values filled, base features created saved to Gold Delta Table!")
print("✅ Ready for PD, LGD, and EAD Modeling!")

✅ All missing values filled, base features created saved to Gold Delta Table!
✅ Ready for PD, LGD, and EAD Modeling!


## Gold Delta Table

Finally, to produce a Gold Delta Table, I will need to sort the dataset in chronological order.

For credit risk modeling, banks use past data loan data to predict future defaults / metrics. As such, we want our dataset to be sorted in **chronological order**, so that built models are trained on older data, and tested on newer data **(out-of-time split)**.

There should not be random splitting of data **(out-of-sample split)**, e.g. `train-test-split` from `sklearn` since credit-risk modeling is a **time-series problem**.

Hence, I will be sorting the dataset right from the start.

By producing the Gold Delta Table, the subsequent jobs would require data scientists to impute missing values, conduct feature engineering and dimensionality reduction for accurate credit risk modeling.


In [20]:
silver_table2 = spark.read.format("delta").load(
    "../data/silver/lendingclub_cleaned_numeric"
)

gold_df = silver_table2.orderBy(["issue_d"], ascending=True)

In [21]:
# == Save as Silver Delta Table 2 (Cleaned Strings Version) ==
gold_df.write.format("delta").mode("overwrite").save(
    "../data/gold/medallion_cleaned_lc_data"
)